# Tetris player

an RL model that can play Tetris game from (https://play.tetris.com/)

In [2]:
# !uv add pandas numpy matplotlib seaborn scikit-learn jupyter
# !uv add torch torchvision torchmetrics
# !uv add flask flask-cors

## Client-Side: data send

### Tampermonkey

```javascript
// ==UserScript==
// @name         Remote Executor (HTTP Polling)
// @match        https://play.tetris.com/
// @grant        GM_xmlhttpRequest
// @connect      192.168.0.156
// ==/UserScript==

(function () {
    const SERVER = 'http://192.168.0.156:8766';

    const poll = () => {
        GM_xmlhttpRequest({
            method: 'GET',
            url: SERVER + '/command',
            onload: (response) => {
                if (response.status === 200 && response.responseText.trim()) {
                    const code = response.responseText;

                    // Wrap user code in an async function
                    let result;
                    try {
                        // Use new Function to avoid eval restrictions and enable async
                        const fn = new Function('return (async () => {' + code + '})()');
                        result = fn(); // This returns a Promise
                    } catch (e) {
                        // Syntax error or function creation failed
                        sendResult({ error: '[JS Parse Error] ' + (e.stack || e.toString()) });
                        return;
                    }

                    // Handle Promise resolution
                    Promise.resolve(result)
                        .then(value => {
                            // Serialize result safely
                            try {
                                sendResult({ result: value });
                            } catch (e) {
                                sendResult({ error: '[Serialize Error] ' + (e.stack || e.toString()) });
                            }
                        })
                        .catch(err => {
                            sendResult({ error: String(err) });
                        });
                }
            },
            onerror: (err) => {
                console.error('[RemoteExec] Poll error:', err);
            },
            timeout: 5000
        });
    };

    function sendResult(payload) {
        GM_xmlhttpRequest({
            method: 'POST',
            url: SERVER + '/result',
            data: JSON.stringify(payload),
            headers: { "Content-Type": "application/json" }
        });
    }

    setInterval(poll, 1000); // Faster polling for responsiveness
})();
```

## Server

In [2]:
import flask
import threading
import json
import time

# Shared state (only one command at a time)
current_command = None
current_result = None
result_ready = threading.Event()
command_requested = threading.Event()

PORT = 8766

app = flask.Flask(__name__)

@app.route('/command', methods=['GET'])
def get_command():
    global current_command
    if current_command is not None:
        # Send command and mark as consumed
        response = flask.make_response(current_command)
        response.headers['Content-Type'] = 'text/plain'
        response.headers['Access-Control-Allow-Origin'] = '*'
        current_command = None
        command_requested.set()  # Confirm client picked it up
        return response
    else:
        # No command ready
        response = flask.make_response('', 204)
        response.headers['Access-Control-Allow-Origin'] = '*'
        return response

@app.route('/result', methods=['POST'])
def post_result():
    global current_result
    try:
        data = flask.request.get_json(force=True)
        current_result = data
    except Exception as e:
        current_result = {"error": str(e)}
    result_ready.set()  # Unblock waiting caller
    response = flask.make_response('', 204)
    response.headers['Access-Control-Allow-Origin'] = '*'
    return response

def start_server():
    app.run(host='0.0.0.0', port=PORT, threaded=True, debug=False)

# Start server in background thread
server_thread = threading.Thread(target=start_server, daemon=True)
server_thread.start()

def execute_js(code: str, timeout: float = 10.0):
    """
    Send JavaScript code to the client and wait for result.

    Args:
        code: JavaScript code to execute (string)
        timeout: Max time to wait for result (seconds)

    Returns:
        dict: Result from client, e.g. {"result": ...} or {"error": ...}

    Raises:
        TimeoutError: If no result received within timeout
    """
    global current_command, current_result

    # Ensure no previous command is pending
    if current_command is not None:
        raise RuntimeError("Another command is already pending")

    current_result = None
    result_ready.clear()
    command_requested.clear()
    current_command = code

    # Wait for client to request the command (optional safety)
    if not command_requested.wait(timeout=2.0):
        print("Warning: Client did not fetch command within 2s")

    # Wait for result
    if result_ready.wait(timeout=timeout):
        return current_result
    else:
        raise TimeoutError("Client did not return result within timeout")

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8766
 * Running on http://192.168.0.156:8766
Press CTRL+C to quit
192.168.0.156 - - [09/Nov/2025 10:01:26] "GET /command HTTP/1.1" 204 -


In [8]:
execute_js("return 2 + 2")
# Example usage (uncomment to test)
# if __name__ == "__main__":
#     import time
#     time.sleep(1)  # Let server start
#     try:
#         result = execute_js("2 + 2")
#         print("Result:", result)
#
#         result = execute_js("document.location.href")
#         print("Page URL:", result)
#
#         result = execute_js("alert('Hello from Python!'); 'alert shown'")
#         print("Alert result:", result)
#
#     except Exception as e:
#         print("Error:", e)

192.168.0.156 - - [09/Nov/2025 10:01:40] "GET /command HTTP/1.1" 200 -
192.168.0.156 - - [09/Nov/2025 10:01:41] "POST /result HTTP/1.1" 204 -


{'result': 4}

'__main__'